In [14]:
import os
import shutil
from pathlib import Path
import mne
import numpy as np
import pandas as pd
import logging
import argparse
import yaml

import torch
from torch import Tensor, nn
from torch.types import Device, _size
from torch.nn.parameter import Parameter, UninitializedParameter
from torch.nn import init
from torch.utils.data import Dataset
from torch.utils.data import ConcatDataset
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from collections import OrderedDict
from fvcore.nn import FlopCountAnalysis, parameter_count_table
import matplotlib.pyplot as plt
from sklearn.preprocessing import Normalizer, StandardScaler
%matplotlib notebook

from models.encoder2 import res_encoderS, res_encoderM
from models.classifier import transformer_classifier
from models.EEGNET import EEGNet


In [15]:
logger = logging.getLogger(__name__)  # Use the current module's name
logging.basicConfig(level=logging.INFO)
# logger.setLevel(logging.DEBUG)
handler = logging.StreamHandler()
# formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
# handler.setFormatter(formatter)
logger.addHandler(handler)
acc_example = 0.95  # Replace with your actual accuracy calculation
logger.info(f"Current accuracy: %{acc_example}")  # Log as info
# logger.debug("Current accuracy: %.2f", accuracy)  # Log as info


Current accuracy: %0.95
INFO:__main__:Current accuracy: %0.95


### FLOPs for auto-encoderS + transformer

In [16]:
parser = argparse.ArgumentParser()
parser.add_argument("config_file", metavar="FILE", help="config file")
# parser.add_argument('--run-dir', metavar='DIR', help='run directory')
# parser.add_argument('--pdb', action='store_true', help='pdb')
args = parser.parse_args(args=['configs/encoderS+transformer.yml'])
# args, opts = parser.parse_known_args()
# f = 'configs/eeg_pt.yml'
with open(args.config_file, 'r') as file:
    Configs = yaml.safe_load(file)

In [17]:
class model(nn.Module):
    def __init__(self, input_size: int, n_channels: int, model_hyp: dict, classes: int):
        super(model, self).__init__()
        self.ae = res_encoderS(n_channels=n_channels, groups=n_channels, num_classes=classes, 
                               len_feature=input_size, d_model=model_hyp['d_model'])
#         self.transformer_encoder = transformer_classifier(input_size, n_channels, model_hyp, classes)
        self.transformer_encoder = transformer_classifier(input_size, n_channels, model_hyp, classes)
        
        self.reset_parameters()
        
    def reset_parameters(self):
        r"""Initiate parameters in the model."""
        
        for p in self.parameters():
            if p.dim() > 1:
#                 logger.debug(p.shape)
                nn.init.xavier_uniform_(p)
                    
        for m in self.modules():
#             print(m)
            if isinstance(m, nn.Conv1d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
        
            elif isinstance(m, (nn.LayerNorm, nn.BatchNorm1d)):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
        print('Complete initiate parameters')

    def forward(self, x):
#         z = self.pe(x)
#         z = x.transpose(-1,-2)
        z = self.ae(x)
#         z = torch.flatten(z, 1)
#         y = self.mlp(z)
        y = self.transformer_encoder(z)
        return y
        
classifier_s = model(input_size=Configs['input_size'],
                                        n_channels = Configs['n_channels'],
                                        model_hyp=Configs['model'],
                                        classes=len(Configs['dataset']['classes']))
classifier_s.eval()

Complete initiate parameters


/home/yossi/anaconda3/envs/python38/lib/python3.8/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.num_heads is odd
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


model(
  (ae): AutoEncoder(
    (conv1): Conv1d(19, 76, kernel_size=(64,), stride=(2,), padding=(3,), groups=19, bias=False)
    (avgpool1d): AdaptiveAvgPool1d(output_size=6000)
    (ln1): LayerNorm((6000,), eps=1e-05, elementwise_affine=True)
    (relu): ReLU(inplace=True)
    (avgpool_1): AvgPool1d(kernel_size=(3,), stride=(2,), padding=(1,))
    (layers): ModuleList(
      (0): Sequential(
        (0): BasicBlock(
          (conv1): Conv1d(76, 76, kernel_size=(16,), stride=(1,), padding=(1,), groups=19, bias=False)
          (avgpool_1): AdaptiveAvgPool1d(output_size=3000)
          (ln1): LayerNorm((3000,), eps=1e-05, elementwise_affine=True)
          (relu): ReLU(inplace=True)
        )
        (1): BasicBlock(
          (conv1): Conv1d(76, 76, kernel_size=(16,), stride=(1,), padding=(1,), groups=19, bias=False)
          (avgpool_1): AdaptiveAvgPool1d(output_size=3000)
          (ln1): LayerNorm((3000,), eps=1e-05, elementwise_affine=True)
          (relu): ReLU(inplace=True)
  

In [19]:
data_s = torch.randn(1, 19, 12000)

In [20]:
# out = classifier(data_s)
# # loss = criterion(out, target)
# probabilities = torch.softmax(out, dim=1)  # Apply softmax to get probabilities
# _, predicted = torch.max(probabilities, 1)  # Get the predicted class
flops = FlopCountAnalysis(classifier_s, data_s)
print("FLOPs:", flops.total())
print(parameter_count_table(classifier_s))

transformer_encoder.encoder_layer, transformer_encoder.encoder_layer.dropout, transformer_encoder.encoder_layer.dropout1, transformer_encoder.encoder_layer.dropout2, transformer_encoder.encoder_layer.linear1, transformer_encoder.encoder_layer.linear2, transformer_encoder.encoder_layer.norm1, transformer_encoder.encoder_layer.norm2, transformer_encoder.encoder_layer.self_attn, transformer_encoder.encoder_layer.self_attn.out_proj, transformer_encoder.transformer_encoder.layers.0.self_attn.out_proj


FLOPs: 201997664
| name                                              | #elements or shape   |
|:--------------------------------------------------|:---------------------|
| model                                             | 2.9M                 |
|  ae                                               |  0.3M                |
|   ae.conv1                                        |   4.9K               |
|    ae.conv1.weight                                |    (76, 1, 64)       |
|   ae.ln1                                          |   12.0K              |
|    ae.ln1.weight                                  |    (6000,)           |
|    ae.ln1.bias                                    |    (6000,)           |
|   ae.layers                                       |   0.2M               |
|    ae.layers.0                                    |    21.7K             |
|    ae.layers.1                                    |    16.3K             |
|    ae.layers.2                                    |    44

### FLOPs for auto-encoderM + transformer

### FLOPs for EEGNet_s

In [5]:
parser = argparse.ArgumentParser()
parser.add_argument("config_file", metavar="FILE", help="config file")
# parser.add_argument('--run-dir', metavar='DIR', help='run directory')
# parser.add_argument('--pdb', action='store_true', help='pdb')
args = parser.parse_args(args=['configs/EEGNET.yml'])
# args, opts = parser.parse_known_args()
# f = 'configs/eeg_pt.yml'
with open(args.config_file, 'r') as file:
    Configs = yaml.safe_load(file)

In [6]:
classifier_eeg_s = EEGNet(signal_length=Configs['input_size'],channel=Configs['n_channels'],
               fs=Configs['processing']['frequency'],
                num_class=len(Configs['dataset']['classes'])
               )
classifier_eeg_s.eval()

Complete initiate parameters


EEGNet(
  (conv2d): Conv2d(1, 8, kernel_size=(1, 50), stride=(1, 1), padding=(0, 23))
  (Batch_normalization_1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (Depthwise_conv2D): Conv2d(8, 24, kernel_size=(19, 1), stride=(1, 1), groups=8)
  (Batch_normalization_2): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (Elu): ELU(alpha=1.0)
  (Average_pooling2D_1): AvgPool2d(kernel_size=(1, 4), stride=(1, 4), padding=0)
  (Dropout): Dropout2d(p=0.2, inplace=False)
  (Separable_conv2D_depth): Conv2d(24, 24, kernel_size=(1, 12), stride=(1, 1), padding=(0, 6), groups=24)
  (Separable_conv2D_point): Conv2d(24, 24, kernel_size=(1, 1), stride=(1, 1))
  (Batch_normalization_3): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (Average_pooling2D_2): AvgPool2d(kernel_size=(1, 8), stride=(1, 8), padding=0)
  (Flatten): Flatten(start_dim=1, end_dim=-1)
  (Dense): Linear(in_features=9000, out_features=2,

In [11]:
data_eeg_s = torch.randn(1, 12000, 19)
data_eeg_s= data_eeg_s.unsqueeze(1)

In [12]:
# loss = criterion(out, target)
# probabilities = torch.softmax(out, dim=1)  # Apply softmax to get probabilities
# _, predicted = torch.max(probabilities, 1)  # Get the predicted class
flops = FlopCountAnalysis(classifier_eeg_s, data_eeg_s)
print("FLOPs:", flops.total())
print(parameter_count_table(classifier_eeg_s))

FLOPs: 103624776
| name                            | #elements or shape   |
|:--------------------------------|:---------------------|
| model                           | 19.9K                |
|  conv2d                         |  0.4K                |
|   conv2d.weight                 |   (8, 1, 1, 50)      |
|   conv2d.bias                   |   (8,)               |
|  Batch_normalization_1          |  16                  |
|   Batch_normalization_1.weight  |   (8,)               |
|   Batch_normalization_1.bias    |   (8,)               |
|  Depthwise_conv2D               |  0.5K                |
|   Depthwise_conv2D.weight       |   (24, 1, 19, 1)     |
|   Depthwise_conv2D.bias         |   (24,)              |
|  Batch_normalization_2          |  48                  |
|   Batch_normalization_2.weight  |   (24,)              |
|   Batch_normalization_2.bias    |   (24,)              |
|  Separable_conv2D_depth         |  0.3K                |
|   Separable_conv2D_depth.weight |   (

### FLOPs for EEGNet_m

In [13]:
parser = argparse.ArgumentParser()
parser.add_argument("config_file", metavar="FILE", help="config file")
# parser.add_argument('--run-dir', metavar='DIR', help='run directory')
# parser.add_argument('--pdb', action='store_true', help='pdb')
args = parser.parse_args(args=['configs/EEGNET.yml'])
# args, opts = parser.parse_known_args()
# f = 'configs/eeg_pt.yml'
with open(args.config_file, 'r') as file:
    Configs = yaml.safe_load(file)

In [15]:
classifier_eeg_m = EEGNet(signal_length=24000,channel=Configs['n_channels'],
               fs=Configs['processing']['frequency'],
                num_class=len(Configs['dataset']['classes'])
               )
classifier_eeg_m.eval()

Complete initiate parameters


EEGNet(
  (conv2d): Conv2d(1, 8, kernel_size=(1, 50), stride=(1, 1), padding=(0, 23))
  (Batch_normalization_1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (Depthwise_conv2D): Conv2d(8, 24, kernel_size=(19, 1), stride=(1, 1), groups=8)
  (Batch_normalization_2): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (Elu): ELU(alpha=1.0)
  (Average_pooling2D_1): AvgPool2d(kernel_size=(1, 4), stride=(1, 4), padding=0)
  (Dropout): Dropout2d(p=0.2, inplace=False)
  (Separable_conv2D_depth): Conv2d(24, 24, kernel_size=(1, 12), stride=(1, 1), padding=(0, 6), groups=24)
  (Separable_conv2D_point): Conv2d(24, 24, kernel_size=(1, 1), stride=(1, 1))
  (Batch_normalization_3): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (Average_pooling2D_2): AvgPool2d(kernel_size=(1, 8), stride=(1, 8), padding=0)
  (Flatten): Flatten(start_dim=1, end_dim=-1)
  (Dense): Linear(in_features=18000, out_features=2

In [18]:
data_eeg_m = torch.randn(1, 24000, 19)
data_eeg_m= data_eeg_m.unsqueeze(1)

In [19]:
# loss = criterion(out, target)
# probabilities = torch.softmax(out, dim=1)  # Apply softmax to get probabilities
# _, predicted = torch.max(probabilities, 1)  # Get the predicted class
flops = FlopCountAnalysis(classifier_eeg_m, data_eeg_m)
print("FLOPs:", flops.total())
print(parameter_count_table(classifier_eeg_m))

FLOPs: 207274776
| name                            | #elements or shape   |
|:--------------------------------|:---------------------|
| model                           | 37.9K                |
|  conv2d                         |  0.4K                |
|   conv2d.weight                 |   (8, 1, 1, 50)      |
|   conv2d.bias                   |   (8,)               |
|  Batch_normalization_1          |  16                  |
|   Batch_normalization_1.weight  |   (8,)               |
|   Batch_normalization_1.bias    |   (8,)               |
|  Depthwise_conv2D               |  0.5K                |
|   Depthwise_conv2D.weight       |   (24, 1, 19, 1)     |
|   Depthwise_conv2D.bias         |   (24,)              |
|  Batch_normalization_2          |  48                  |
|   Batch_normalization_2.weight  |   (24,)              |
|   Batch_normalization_2.bias    |   (24,)              |
|  Separable_conv2D_depth         |  0.3K                |
|   Separable_conv2D_depth.weight |   (

### FLOPs for Transformer classifier

In [20]:
parser = argparse.ArgumentParser()
parser.add_argument("config_file", metavar="FILE", help="config file")
# parser.add_argument('--run-dir', metavar='DIR', help='run directory')
# parser.add_argument('--pdb', action='store_true', help='pdb')
args = parser.parse_args(args=['configs/EEGNET.yml'])
# args, opts = parser.parse_known_args()
# f = 'configs/eeg_pt.yml'
with open(args.config_file, 'r') as file:
    Configs = yaml.safe_load(file)

In [29]:
class transformer_classifier(nn.Module):
    def __init__(self, input_size:int, n_channels:int, model_hyp:dict, classes:int):
        super(transformer_classifier, self).__init__()

#         self.encoder_layer = nn.TransformerEncoderLayer(d_model=model_hyp['d_model'],
#                                                         nhead=model_hyp['n_head'], batch_first=True)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=24000,
                                                        nhead=1, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=1)
        self.flatten = nn.Flatten()
#         self.linear = nn.Linear(model_hyp['d_model']*n_channels, classes)
        self.linear = nn.Linear(24000*n_channels, classes)
    
    def forward(self, x):
        z = self.transformer_encoder(x)
        logger.debug(f"transformer output size: {z.shape}")
        z = self.flatten(z)
        logger.debug(f"flatten output size: {z.shape}")
        y = self.linear(z)
        logger.debug(f"linear output size: {y.shape}")
        return y

In [30]:
transformer_s= transformer_classifier(24000, 19, 12000, 2)
transformer_s.eval()
data_s = torch.randn(1, 19,12000)

/home/yossi/anaconda3/envs/python38/lib/python3.8/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.num_heads is odd
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [31]:
# loss = criterion(out, target)
# probabilities = torch.softmax(out, dim=1)  # Apply softmax to get probabilities
# _, predicted = torch.max(probabilities, 1)  # Get the predicted class
flops = FlopCountAnalysis(transformer_s, data_s)
print("FLOPs:", flops.total())
print(parameter_count_table(transformer_s))

encoder_layer, encoder_layer.dropout, encoder_layer.dropout1, encoder_layer.dropout2, encoder_layer.linear1, encoder_layer.linear2, encoder_layer.norm1, encoder_layer.norm2, encoder_layer.self_attn, encoder_layer.self_attn.out_proj, transformer_encoder.layers.0.self_attn.out_proj


FLOPs: 45649248000
| name                                      | #elements or shape   |
|:------------------------------------------|:---------------------|
| model                                     | 4.8G                 |
|  encoder_layer                            |  2.4G                |
|   encoder_layer.self_attn                 |   2.3G               |
|    encoder_layer.self_attn.in_proj_weight |    (72000, 24000)    |
|    encoder_layer.self_attn.in_proj_bias   |    (72000,)          |
|    encoder_layer.self_attn.out_proj       |    0.6G              |
|   encoder_layer.linear1                   |   49.2M              |
|    encoder_layer.linear1.weight           |    (2048, 24000)     |
|    encoder_layer.linear1.bias             |    (2048,)           |
|   encoder_layer.linear2                   |   49.2M              |
|    encoder_layer.linear2.weight           |    (24000, 2048)     |
|    encoder_layer.linear2.bias             |    (24000,)          |
|   encoder_lay

### EEG-ARNN

In [1]:
#!/usr/bin/env python
# coding: utf-8

import os
import math
import pandas as pd
import numpy as np
from pathlib import Path
import shutil
import argparse
from typing import List, Union
import matplotlib.pyplot as plt
import yaml
from datetime import datetime
import logging
import time

from sklearn.metrics import confusion_matrix

import torch
from torch import Tensor, nn
from torch.types import Device, _size
from torch.nn.parameter import Parameter, UninitializedParameter
from torch.nn import init
from torch.utils.data import Dataset
from torch.utils.data import ConcatDataset
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from collections import OrderedDict

from configs.config import configs

# from models.pe import PositionalEncoding
from EEG_ARNN.gcnModelST_pytorch import GCN_layer
from EEG_ARNN.nnModelST_pytorch import zhnn

from fvcore.nn import FlopCountAnalysis, parameter_count_table

2024-12-09 16:59:57.665662: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-09 16:59:57.687386: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-09 16:59:58.057806: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
parser = argparse.ArgumentParser()
parser.add_argument("config_file", metavar="FILE", help="config file")
# parser.add_argument('--run-dir', metavar='DIR', help='run directory')
# parser.add_argument('--pdb', action='store_true', help='pdb')
args = parser.parse_args(args=['configs/EEG-ARNN.yml'])
# args, opts = parser.parse_known_args()
# f = 'configs/eeg_pt.yml'
with open(args.config_file, 'r') as file:
    Configs = yaml.safe_load(file)

In [5]:
def preprocess_adj(adj):
    adj = adj + np.eye(adj.shape[0])
    adj = normalize_adj(adj)
    return adj

def normalize_adj(adj):
    d = np.diag(np.power(np.array(adj.sum(1)), -0.5).flatten(), 0)
    a_norm = adj.dot(d).transpose().dot(d)
    return a_norm

df = pd.read_excel('./configs/init_adj_tuh.xlsx')
Abf = df.iloc[:, 1:].values
A = preprocess_adj(Abf)
# A = np.ones((60,60))
A = np.float32(A)
A = torch.from_numpy(A).to('cpu')

model = zhnn((19,12000), A).to('cpu')
x = torch.rand(1, 1, 19, 12000)

flops = FlopCountAnalysis(model, x)
print("FLOPs:", flops.total())
print(parameter_count_table(model))

Unsupported operator aten::pad encountered 3 time(s)
Unsupported operator aten::add_ encountered 7 time(s)
Unsupported operator aten::elu_ encountered 7 time(s)
Unsupported operator aten::feature_dropout encountered 7 time(s)
Unsupported operator aten::add encountered 3 time(s)
Unsupported operator aten::avg_pool2d encountered 3 time(s)


FLOPs: 260259000.0
| name             | #elements or shape   |
|:-----------------|:---------------------|
| model            | 69.8K                |
|  A               |  (19, 19)            |
|  conv1           |  0.3K                |
|   conv1.weight   |   (16, 1, 1, 16)     |
|   conv1.bias     |   (16,)              |
|  norm1           |  32                  |
|   norm1.weight   |   (16,)              |
|   norm1.bias     |   (16,)              |
|  gconv2          |  27.6K               |
|   gconv2.W       |   (60, 60)           |
|   gconv2.theta   |   (12000,)           |
|   gconv2.b       |   (1, 1, 1, 12000)   |
|  norm2           |  32                  |
|   norm2.weight   |   (16,)              |
|   norm2.bias     |   (16,)              |
|  dconv3          |  0.1K                |
|   dconv3.weight  |   (16, 1, 1, 8)      |
|   dconv3.bias    |   (16,)              |
|  norm3           |  32                  |
|   norm3.weight   |   (16,)              |
|   norm3.bia

### FLOPs for Deep4Conv

In [6]:
import os
import math
import pandas as pd
import numpy as np
from pathlib import Path
import shutil
import argparse
from typing import List, Union
import matplotlib.pyplot as plt
import yaml
from datetime import datetime
import logging
import time

from sklearn.metrics import confusion_matrix

import torch
from torch import Tensor, nn
from torch.types import Device, _size
from torch.nn.parameter import Parameter, UninitializedParameter
from torch.nn import init
from torch.utils.data import Dataset
from torch.utils.data import ConcatDataset
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from collections import OrderedDict

from configs.config import configs


from braindecode.models import Deep4Net

from fvcore.nn import FlopCountAnalysis, parameter_count_table

/home/yossi/.local/lib/python3.8/site-packages/transformers/utils/generic.py:482: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [7]:
parser = argparse.ArgumentParser()
parser.add_argument("config_file", metavar="FILE", help="config file")
# parser.add_argument('--run-dir', metavar='DIR', help='run directory')
# parser.add_argument('--pdb', action='store_true', help='pdb')
args = parser.parse_args(args=['configs/Deep4Conv.yml'])
# args, opts = parser.parse_known_args()
# f = 'configs/eeg_pt.yml'
with open(args.config_file, 'r') as file:
    Configs = yaml.safe_load(file)

In [8]:
class model(nn.Module):
    def __init__(self, input_size: int, n_channels: int, model_hyp: dict, classes: int):
        super(model, self).__init__()
        self.de = Deep4Net(in_chans=n_channels, n_classes=classes,
                 input_window_samples=input_size,
                 n_filters_time=25, n_filters_spat=25,
                 stride_before_pool=True,
                 n_filters_2=n_channels * 2,
                 n_filters_3=n_channels * 4,
                 n_filters_4=n_channels * 8,
                 final_conv_length=1,
                 add_log_softmax=False, )
        self.linear = nn.Linear(2*437, classes)
        
        self.reset_parameters()
        
    def reset_parameters(self):
        r"""Initiate parameters in the model."""
        
        for p in self.parameters():
            if p.dim() > 1:
#                 logger.debug(p.shape)
                nn.init.xavier_uniform_(p)
                    
        for m in self.modules():
#             print(m)
            if isinstance(m, nn.Conv1d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
        
            elif isinstance(m, (nn.LayerNorm, nn.BatchNorm2d)):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
        print('Complete initiate parameters')

    def forward(self, x):
#         z = self.pe(x)
        z = x.transpose(-1,-2)
        z = self.de(z)
        z = torch.flatten(z, 1)
        y = self.linear(z)
        return y

In [9]:
model = model(input_size=Configs['input_size'],
                                        n_channels = Configs['n_channels'],
                                        model_hyp=Configs['model'],
                                        classes=len(Configs['dataset']['classes']))

Complete initiate parameters


/home/yossi/anaconda3/envs/python38/lib/python3.8/site-packages/braindecode/models/base.py:23: UserWarning: Deep4Net: 'in_chans' is depreciated. Use 'n_chans' instead.
  warnings.warn(
/home/yossi/anaconda3/envs/python38/lib/python3.8/site-packages/braindecode/models/base.py:23: UserWarning: Deep4Net: 'n_classes' is depreciated. Use 'n_outputs' instead.
  warnings.warn(
/home/yossi/anaconda3/envs/python38/lib/python3.8/site-packages/braindecode/models/base.py:23: UserWarning: Deep4Net: 'input_window_samples' is depreciated. Use 'n_times' instead.
  warnings.warn(


In [11]:
x = torch.rand(1, 12000, 19)

flops = FlopCountAnalysis(model, x)
print("FLOPs:", flops.total())
print(parameter_count_table(model))

Unsupported operator aten::mul encountered 4 time(s)
Unsupported operator aten::sum encountered 2 time(s)
Unsupported operator aten::add encountered 1 time(s)
Unsupported operator aten::elu encountered 4 time(s)
Unsupported operator aten::max_pool2d encountered 4 time(s)
The following submodules of the model were never called during the trace of the graph. They may be unused, or they were accessed by direct calls to .forward() or via other python methods. In the latter case they will have zeros for statistics, though their statistics will still contribute to their parent calling module.
de.conv_time_spat.conv_spat, de.conv_time_spat.conv_time, de.pool_nonlin, de.pool_nonlin_2, de.pool_nonlin_3, de.pool_nonlin_4


FLOPs: 185339797
| name                              | #elements or shape   |
|:----------------------------------|:---------------------|
| model                             | 0.2M                 |
|  de                               |  0.2M                |
|   de.conv_time_spat               |   12.2K              |
|    de.conv_time_spat.conv_time    |    0.3K              |
|    de.conv_time_spat.conv_spat    |    11.9K             |
|   de.bnorm                        |   50                 |
|    de.bnorm.weight                |    (25,)             |
|    de.bnorm.bias                  |    (25,)             |
|   de.conv_2                       |   9.5K               |
|    de.conv_2.weight               |    (38, 25, 10, 1)   |
|   de.bnorm_2                      |   76                 |
|    de.bnorm_2.weight              |    (38,)             |
|    de.bnorm_2.bias                |    (38,)             |
|   de.conv_3                       |   28.9K              |
|    de

### FLOPs for FusionCNN

In [ ]:
from fvcore.nn import FlopCountAnalysis, parameter_count_table